# Assignment 3A: Frequent Patterns - Individual code

\Alex Nguyen

**Business Scenario**

You are working in a team of developers for a grocery store. The store manager noticed that some items are often bought together. The manager wants to find out exactly what items customers buy most often together in one basket (we call them itemsets). This information will be used to place itemsets close together, so that customers can find them quickly, which in turn may increase sales.

After analysing the problem, your team has discovered that once frequent itemsets are identified, it is also possible to recommend products from these itemsets to customers on the store website.

Your team, being knowledgeable of both frequent itemsets mining and recommendation systems, wants to go even further: you want to test other well-known recommendation methods, such as collaborative filtering, to see which recommendation method works better.

The store collects details about customers’ buying habits through a loyalty programme and your team is given access to the representative dataset. The system you build, however, should scale to around one million customer transactions.

The project has been approved by the store management, so you are ready to start building the system which can help to significantly increase sales.

## Task 1: Patterns

Write code in Python to mine frequent patterns from the training dataset with pattern mining algorithm discussed in the course - FP-growth.

At the end of this task, you should have a system which can take raw input data and produce a series of patterns (and some kind of good/bad quality of the patterns).

### 0. Install & load library

In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

### 1. Load data

In [2]:
train_data = pd.read_csv('Groceries data train.csv', header=0)
train_data

,Member_number,Date,itemDescription,year,month,day,day_of_week
0,3021,30/01/2015,frankfurter,2015,1,30,4
1,1292,24/10/2015,pork,2015,10,24,5
2,4206,4/04/2014,root vegetables,2014,4,4,4
3,4369,25/08/2015,onions,2015,8,25,1
4,1522,1/07/2014,waffles,2014,7,1,1
...,...,...,...,...,...,...,...
26995,4206,14/01/2014,rolls/buns,2014,1,14,1
26996,1660,15/06/2014,domestic eggs,2014,6,15,6
26997,1595,10/05/2015,liver loaf,2015,5,10,6
26998,3231,31/03/2015,sausage,2015,3,31,1


### 2. Check itemsets

In [3]:
# check type of data
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27000 entries, 0 to 26999
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    27000 non-null  int64 
 1   Date             27000 non-null  object
 2   itemDescription  27000 non-null  object
 3   year             27000 non-null  int64 
 4   month            27000 non-null  int64 
 5   day              27000 non-null  int64 
 6   day_of_week      27000 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 1.4+ MB


In [4]:
# check data value
train_data.describe()

,Member_number,year,month,day,day_of_week
count,27000.000000,27000.000000,27000.000000,27000.000000,27000.000000
mean,3002.872667,2014.529037,6.485074,15.744444,3.020852
std,1153.928123,0.499165,3.431239,8.784553,1.987826
min,1000.000000,2014.000000,1.000000,1.000000,0.000000
25%,2003.000000,2014.000000,4.000000,8.000000,1.000000
50%,3002.000000,2015.000000,6.000000,16.000000,3.000000
75%,4010.000000,2015.000000,9.000000,23.000000,5.000000
max,5000.000000,2015.000000,12.000000,31.000000,6.000000


In [5]:
# check item
train_data['itemDescription'].unique()

array(['frankfurter', 'pork', 'root vegetables', 'onions', 'waffles',
       'cereals', 'yogurt', 'sausage', 'rolls/buns', 'chocolate',
       'whipped/sour cream', 'butter', 'curd', 'frozen meals',
       'whole milk', 'tropical fruit', 'sugar', 'specialty cheese',
       'other vegetables', 'fruit/vegetable juice', 'misc. beverages',
       'frozen vegetables', 'spices', 'coffee', 'cream cheese ',
       'red/blush wine', 'dessert', 'long life bakery product',
       'domestic eggs', 'abrasive cleaner', 'berries',
       'liquor (appetizer)', 'semi-finished bread', 'newspapers',
       'beverages', 'beef', 'female sanitary products', 'salt', 'soda',
       'sauces', 'oil', 'snack products', 'brown bread',
       'hygiene articles', 'artif. sweetener', 'bottled beer',
       'canned beer', 'hamburger meat', 'liver loaf', 'soups',
       'pip fruit', 'hard cheese', 'shopping bags', 'canned vegetables',
       'napkins', 'citrus fruit', 'margarine', 'pasta', 'salty snack',
       'candl

In [6]:
# preprocess itemdescription

# to lower string
train_data["itemDescription"] = train_data["itemDescription"].str.lower()

# remove special characters/punctuation
train_data["itemDescription"] = train_data["itemDescription"].str.replace(r'[^\w\s]+', '', regex=True)

train_data['itemDescription'].unique()

array(['frankfurter', 'pork', 'root vegetables', 'onions', 'waffles',
       'cereals', 'yogurt', 'sausage', 'rollsbuns', 'chocolate',
       'whippedsour cream', 'butter', 'curd', 'frozen meals',
       'whole milk', 'tropical fruit', 'sugar', 'specialty cheese',
       'other vegetables', 'fruitvegetable juice', 'misc beverages',
       'frozen vegetables', 'spices', 'coffee', 'cream cheese ',
       'redblush wine', 'dessert', 'long life bakery product',
       'domestic eggs', 'abrasive cleaner', 'berries', 'liquor appetizer',
       'semifinished bread', 'newspapers', 'beverages', 'beef',
       'female sanitary products', 'salt', 'soda', 'sauces', 'oil',
       'snack products', 'brown bread', 'hygiene articles',
       'artif sweetener', 'bottled beer', 'canned beer', 'hamburger meat',
       'liver loaf', 'soups', 'pip fruit', 'hard cheese', 'shopping bags',
       'canned vegetables', 'napkins', 'citrus fruit', 'margarine',
       'pasta', 'salty snack', 'candles', 'cat food',

In [7]:
# convert date to datestamp
train_data['Date'] = pd.to_datetime(train_data['Date'], format="%d/%m/%Y")
train_data

,Member_number,Date,itemDescription,year,month,day,day_of_week
0,3021,2015-01-30,frankfurter,2015,1,30,4
1,1292,2015-10-24,pork,2015,10,24,5
2,4206,2014-04-04,root vegetables,2014,4,4,4
3,4369,2015-08-25,onions,2015,8,25,1
4,1522,2014-07-01,waffles,2014,7,1,1
...,...,...,...,...,...,...,...
26995,4206,2014-01-14,rollsbuns,2014,1,14,1
26996,1660,2014-06-15,domestic eggs,2014,6,15,6
26997,1595,2015-05-10,liver loaf,2015,5,10,6
26998,3231,2015-03-31,sausage,2015,3,31,1


The dataset presents the item purchases of each member from 01/01/2014 to 30/12/2015.

For effective pattern mining, it is recommended to aggregate the list of items purchased by each member into quarterly periods each year. The reasons behind are [1]:

* Captures seasonal variations and holiday impacts on consumer behavior

* Aligns with business quarters for effective strategic reviews and planning of storing products

In [8]:
# extract year and date to quarter
train_data['Year'] = train_data['Date'].dt.year
train_data['Quarter'] = train_data['Date'].dt.to_period('Q')

In [9]:
# group data within 1 transaction based on member number, year, and quarter
grouped = train_data.groupby(['Member_number', 'Year', 'Quarter'])['itemDescription'].apply(list).reset_index()
grouped

,Member_number,Year,Quarter,itemDescription
0,1000,2014,2014Q2,[pastry]
1,1000,2015,2015Q1,"[sausage, yogurt]"
2,1000,2015,2015Q2,"[soda, pickled vegetables]"
3,1000,2015,2015Q3,"[misc beverages, canned beer]"
4,1000,2015,2015Q4,[sausage]
...,...,...,...,...
11307,4999,2015,2015Q2,[butter milk]
11308,4999,2015,2015Q4,"[herbs, bottled water]"
11309,5000,2014,2014Q3,"[onions, fruitvegetable juice]"
11310,5000,2014,2014Q4,[other vegetables]


In [10]:
# extract item lists for the FP-Growth input
transactions = grouped['itemDescription'].tolist()
transactions

[['pastry'],
 ['sausage', 'yogurt'],
 ['soda', 'pickled vegetables'],
 ['misc beverages', 'canned beer'],
 ['sausage'],
 ['rollsbuns', 'whole milk', 'sausage'],
 ['soda', 'whole milk'],
 ['frankfurter', 'soda', 'whippedsour cream', 'curd'],
 ['white bread', 'beef'],
 ['whole milk', 'butter'],
 ['frozen vegetables'],
 ['sugar', 'tropical fruit'],
 ['butter milk'],
 ['dental care', 'rollsbuns'],
 ['root vegetables', 'detergent'],
 ['sausage'],
 ['canned beer', 'cling filmbags', 'dish cleaner', 'frozen fish'],
 ['pip fruit', 'whole milk'],
 ['whole milk', 'pastry'],
 ['shopping bags', 'chocolate', 'redblush wine', 'whole milk', 'rollsbuns'],
 ['other vegetables', 'hygiene articles'],
 ['whippedsour cream', 'margarine'],
 ['rollsbuns', 'rollsbuns'],
 ['softener'],
 ['rice'],
 ['shopping bags'],
 ['chicken',
  'bottled beer',
  'frankfurter',
  'rollsbuns',
  'chocolate',
  'frankfurter',
  'flour'],
 ['dessert', 'hamburger meat', 'liver loaf'],
 ['photofilm', 'tropical fruit', 'white wine'

### 3. Prepare data

In [11]:
# transaction encoder initialisation
encoder = TransactionEncoder()
encoder

TransactionEncoder()

In [12]:
# convert to a format for frequent itemset mining
encoded_array = encoder.fit_transform(transactions)
encoded_array

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False,  True, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [13]:
# create dataframe
df = pd.DataFrame(encoded_array, columns=encoder.columns_)
df

,abrasive cleaner,artif sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,beverages,bottled beer,...,uhtmilk,vinegar,waffles,whippedsour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11307,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11308,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11309,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11310,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### 4. Apply FP-Growth algorithm to train data

The system requires scaling up to 1 million customer transactions. The Frequent Patterns (FP) Growth algorithm is recommended because many grocery transactions likely follow the same path, allowing for high data compaction. Thus, the FP-Growth algorithm is more efficient, providing the same results as the Apriori algorithm with improved performance. [2]

The recommended parameter values are as follows: [5]

* For 1,000,000 transactions, set the minimum support to 0.001. This considers only the itemsets that occur at least 1,000 times in total transactions.

* For handling 1,000 frequent itemsets, set the minimum confidence to 0.1. This considers the rules where the items appear together with the given itemsets more than 10% of the time.

* Consider only the association rules that increase the likelihood of purchasing items together - a lift value greater than 1 is recommended to help store managers increase sales.

In [14]:
# create frequent itemsets
frequent_itemsets = fpgrowth(df, min_support=0.001, use_colnames=True) # 
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)

print("Frequent itemsets:\n", frequent_itemsets)

Frequent itemsets:
       support                        itemsets
7    0.141089                    (whole milk)
29   0.111386              (other vegetables)
8    0.100690                     (rollsbuns)
3    0.089905                          (soda)
1    0.078324                        (yogurt)
..        ...                             ...
378  0.001061     (chocolate, tropical fruit)
123  0.001061                   (ready soups)
406  0.001061      (domestic eggs, margarine)
667  0.001061           (salty snack, yogurt)
728  0.001061  (whole milk, liquor appetizer)

[729 rows x 2 columns]


In [23]:
# generate association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
rules = rules[rules['lift'] > 1]

if not rules.empty:
    print("Association rules:\n", rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])
else:
    print("No rules generated.")

Association rules:
                    antecedents   consequents   support  confidence      lift
0                 (whole milk)   (rollsbuns)  0.014940    0.105890  1.051646
1                  (rollsbuns)  (whole milk)  0.014940    0.148376  1.051646
4                     (yogurt)  (whole milk)  0.011227    0.143341  1.015960
6                    (sausage)  (whole milk)  0.009017    0.165584  1.173616
7              (bottled water)  (whole milk)  0.008310    0.144838  1.026573
..                         ...           ...       ...         ...       ...
182       (pickled vegetables)   (rollsbuns)  0.001061    0.127660  1.267853
183         (processed cheese)      (yogurt)  0.001061    0.105263  1.343947
184        (soda, canned beer)  (whole milk)  0.001061    0.307692  2.180837
185  (canned beer, whole milk)        (soda)  0.001061    0.166667  1.853818
186         (liquor appetizer)  (whole milk)  0.001061    0.266667  1.890058

[122 rows x 5 columns]


### 5. Apply association rules to test data

In [16]:
# load test data
test_data = pd.read_csv('Groceries data test.csv', header=0)

In [17]:
# check item & preprocess item

# to lower string
test_data["itemDescription"] = test_data["itemDescription"].str.lower()

# remove special characters/punctuation
test_data["itemDescription"] = test_data["itemDescription"].str.replace(r'[^\w\s]+', '', regex=True)


In [18]:
# group data
test_data['Date'] = pd.to_datetime(test_data['Date'], format="%d/%m/%Y")
test_data['Year'] = test_data['Date'].dt.year
test_data['Quarter'] = test_data['Date'].dt.to_period('Q')
test_grouped = test_data.groupby(['Member_number', 'Year', 'Quarter'])['itemDescription'].apply(list).reset_index()
test_grouped

,Member_number,Year,Quarter,itemDescription
0,1000,2014,2014Q2,"[whole milk, salty snack]"
1,1000,2015,2015Q1,"[semifinished bread, whole milk]"
2,1000,2015,2015Q4,[hygiene articles]
3,1001,2015,2015Q1,[frankfurter]
4,1002,2014,2014Q3,[other vegetables]
...,...,...,...,...
7683,4999,2014,2014Q2,[detergent]
7684,4999,2015,2015Q1,"[berries, yogurt]"
7685,4999,2015,2015Q2,[whippedsour cream]
7686,5000,2014,2014Q4,[bottled beer]


In [19]:
# convert to transaction data
test_transactions = test_grouped['itemDescription'].tolist()
test_transactions

[['whole milk', 'salty snack'],
 ['semifinished bread', 'whole milk'],
 ['hygiene articles'],
 ['frankfurter'],
 ['other vegetables'],
 ['specialty chocolate'],
 ['frozen meals', 'rollsbuns'],
 ['rollsbuns'],
 ['tropical fruit'],
 ['root vegetables'],
 ['rollsbuns', 'other vegetables', 'chocolate', 'packaged fruitvegetables'],
 ['bottled water', 'rollsbuns'],
 ['whole milk'],
 ['skin care'],
 ['whole milk'],
 ['liquor appetizer'],
 ['soda', 'yogurt', 'root vegetables'],
 ['newspapers', 'yogurt'],
 ['ketchup'],
 ['frankfurter'],
 ['rollsbuns', 'candles'],
 ['yogurt'],
 ['root vegetables'],
 ['mustard', 'hard cheese', 'whippedsour cream', 'napkins', 'tropical fruit'],
 ['white bread', 'roll products ', 'whole milk'],
 ['whole milk', 'rollsbuns', 'hamburger meat'],
 ['canned beer'],
 ['rollsbuns'],
 ['salty snack'],
 ['citrus fruit'],
 ['uhtmilk'],
 ['oil', 'chicken'],
 ['yogurt'],
 ['soda', 'shopping bags'],
 ['berries'],
 ['butter milk'],
 ['redblush wine'],
 ['whole milk'],
 ['soda', '

In [20]:
# encode test transactions
encoded_array_test = encoder.transform(test_transactions)
df_test = pd.DataFrame(encoded_array_test, columns=encoder.columns_)
df_test

,abrasive cleaner,artif sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,beverages,bottled beer,...,uhtmilk,vinegar,waffles,whippedsour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7683,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7684,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
7685,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
7686,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [24]:
# function to apply rules to test data
def apply_rules(transactions, rules):

    results = []
    for _, rule in rules.iterrows():

        antecedents = set(rule['antecedents'])
        print(f"{antecedents} -> {set(rule['consequents'])}")
        check_rule = transactions.apply(lambda x: antecedents.issubset(set(x.index[x])), axis=1)

        if check_rule.any():
            results.append({
                'rule': f"{set(rule['antecedents'])} -> {set(rule['consequents'])}",
                'support': rule['support'],
                'confidence': rule['confidence'],
                'lift': rule['lift'],
                'coverage': check_rule.sum()
            })

    return pd.DataFrame(results)


# apply rules to test data
rule_applications = apply_rules(df_test, rules)

print("\nUpdated rule applications:\n", rule_applications)

{'whole milk'} -> {'rollsbuns'}
{'rollsbuns'} -> {'whole milk'}
{'yogurt'} -> {'whole milk'}
{'sausage'} -> {'whole milk'}
{'bottled water'} -> {'whole milk'}
{'bottled beer'} -> {'whole milk'}
{'canned beer'} -> {'whole milk'}
{'butter'} -> {'whole milk'}
{'sausage'} -> {'soda'}
{'bottled water'} -> {'rollsbuns'}
{'citrus fruit'} -> {'other vegetables'}
{'frankfurter'} -> {'whole milk'}
{'shopping bags'} -> {'rollsbuns'}
{'frankfurter'} -> {'other vegetables'}
{'bottled beer'} -> {'other vegetables'}
{'fruitvegetable juice'} -> {'whole milk'}
{'shopping bags'} -> {'soda'}
{'pip fruit'} -> {'rollsbuns'}
{'bottled beer'} -> {'rollsbuns'}
{'frozen vegetables'} -> {'whole milk'}
{'pork'} -> {'other vegetables'}
{'domestic eggs'} -> {'other vegetables'}
{'brown bread'} -> {'rollsbuns'}
{'frankfurter'} -> {'soda'}
{'white bread'} -> {'whole milk'}
{'pork'} -> {'rollsbuns'}
{'chicken'} -> {'rollsbuns'}
{'frozen vegetables'} -> {'other vegetables'}
{'fruitvegetable juice'} -> {'rollsbuns'}
{'

In [25]:
# evaluate the rules

if not rule_applications.empty:

    # calculate overall coverage
    total_transactions = df_test.shape[0]
    covered_transactions = rule_applications['coverage'].sum()
    coverage_percentage = (covered_transactions / total_transactions) * 100

    # calculate average support, confidence, and lift for applicable rules
    avg_support = rule_applications['support'].mean()
    avg_confidence = rule_applications['confidence'].mean()
    avg_lift = rule_applications['lift'].mean()

    print("Total transactions in test set:", total_transactions)
    print("Transactions covered by rules:", covered_transactions)
    print("Coverage percentage:", coverage_percentage)
    print("Average support of applicable rules:", avg_support)
    print("Average confidence of applicable rules:", avg_confidence)
    print("Average lift of applicable rules:", avg_lift)
    
    # detailed insights into rules with significant coverage
    significant_rules = rule_applications[rule_applications['coverage'] > (0.01 * total_transactions)] # at least 1% total transaction - 10,000 transactions if scaled up
    print("\nDetailed rules with significant coverage:\n", significant_rules)

else:
    print("No rules were applicable to the test transactions.")

Total transactions in test set: 7688
Transactions covered by rules: 11317
Coverage percentage: 147.20343392299688
Average support of applicable rules: 0.002677412062049297
Average confidence of applicable rules: 0.14519040999964752
Average lift of applicable rules: 1.3292264940592076

Detailed rules with significant coverage:
                                              rule   support  confidence  \
0                 {'whole milk'} -> {'rollsbuns'}  0.014940    0.105890   
1                 {'rollsbuns'} -> {'whole milk'}  0.014940    0.148376   
2                    {'yogurt'} -> {'whole milk'}  0.011227    0.143341   
3                   {'sausage'} -> {'whole milk'}  0.009017    0.165584   
4             {'bottled water'} -> {'whole milk'}  0.008310    0.144838   
5              {'bottled beer'} -> {'whole milk'}  0.006807    0.159420   
6               {'canned beer'} -> {'whole milk'}  0.006365    0.150313   
7                    {'butter'} -> {'whole milk'}  0.006011    0.181333

## References

[1] Chaudhary, S 2024, *'Market basket analysis: anticipating customer behavior'*, webpage, Turing, viewed 25 May 2024, <https://www.turing.com/kb/market-basket-analysis>.

[2] Kotu, V 2019, *Data science: concepts and practice*, B Deshpande (ed.), Second edition., Morgan Kaufmann Publishers an imprint of Elsevier, Cambridge, pp. 1-570.

[3] University of Adelaide 2024-1, COMP_SCI_7306 Mining Big Data - Module 9 Lecture: Association, pdf, The University of Adelaide, viewed 25 May 2024, <https://myuni.adelaide.edu.au/courses/91968/files/14319270?module_item_id=3274647>.

[4] University of Adelaide 2024-2, COMP_SCI_7306 Mining Big Data - Module 10 Workshop: Association Patterns, Python notebook, The University of Adelaide, viewed 25 May 2024, <https://myuni.adelaide.edu.au/courses/91968/files/14319255?wrap=1>.

[5] Garg, A 2018, *'Complete guide to Association Rules'*, webpage, Towards Data Science, viewed 25 May 2024, <https://towardsdatascience.com/association-rules-2-aa9a77241654>.

